In [127]:
from typing import List

from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
from bs4.element import Tag

def create_url(zipcode: int, sort_by: str=None, verbose: bool=False) -> str:
    """
    Generates a URL to the Zillow properties listed
    for the given zipcode.
    
    Parameters
    ---------
    zipcode : int
        The zipcode to generate the URL for
        
    sort_by : str, optional; default = None; options: {
    'high-low', 'low-high', 'newest', 'bedrooms', 'bathrooms',
    'house-size', 'lot-size', 'zest-high-low', 'zest-low-high'
    }
        Sort the listings by the given filter.
        
    Returns
    -------
    url : str
    """
    sort_options = {
        'high-low': 'priced_sort',
        'low-high': 'pricea_sort',
        'newest': 'days_sort',
        'bedrooms': 'beds_sort',
        'bathrooms': 'baths_sort',
        'house-size': 'size_sort',
        'lot-size': 'lot_sort',
        'zest-high-low': 'zest_sort',
        'zest-low-high': 'zesta_sort'
    }
    
    option = sort_options.get(sort_by, 'globalrelevanceex_sort')
    url = f'https://www.zillow.com/homes/for_sale/{zipcode}/0_singlestory/{option}'

    if verbose:
        print(url)

    return url

def get_zpids(zipcode: int, sort_by: str=None, verbose: bool=False) -> List[int]:
    """
    Extracts all Zillow Property ID's (zpid) for each listing
    in the given zipcode.
    
    Parameters
    ---------
    zipcode : int
        The zipcode to retrieve the ZPID's for
        
    sort_by : str, optional; default = None; options: {
        'high-low', 'low-high', 'newest', 'bedrooms', 'bathrooms',
        'house-size', 'lot-size', 'zest-high-low', 'zest-low-high'
    }
        Sort the listings by the given filter.
        
    Returns
    -------
    zpids : List[int]
    
    """
    url = create_url(zipcode, sort_by, verbose)
    # TODO replace with get_headers()
    headers = {'User-Agent': 'Mozilla/5.0'}
    headers = get_headers()

    req = Request(url, headers=headers)
    webpage = urlopen(req).read()
    # parse the HTML into a tree
    soup = BeautifulSoup(webpage)
    
    listings = soup.find(
        'ul', 
        {'class': 'photo-cards photo-cards_wow photo-cards_short'}
    )
    
    def _extract_zpid(li: Tag) -> int:
        article = li.find(
            'article',
            {'class': 'list-card list-card-short list-card_not-saved'}
        )
        if article is None:
            return None
        
        return int(article.get('id').replace('zpid_', ''))

    zpids = []
    for li in listings.find_all('li'):
        zpid = _extract_zpid(li)
        if zpid is not None:
            zpids.append(zpid)
        
    return zpids
    

# FIXME - causing incorrect encoding
def get_headers():
    headers = {
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'accept-encoding': 'gzip, deflate, sdch, br',
        'accept-language': 'en-GB,en;q=0.8,en-US;q=0.6,ml;q=0.4',
        'cache-control': 'max-age=0',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.5304.87 Safari/537.36',
        #'user-agent': 'Chrome/107.0.5304.87'
    }
    return headers
# can then easily turn this into a CLI
if __name__ == '__main__':
    zipcode = 78254  # San Antonio, Far West Side
    zpids = get_zpids(zipcode, sort_by='newest')

AttributeError: 'Request' object has no attribute 'text'

In [66]:
import bs4 , requests

res = requests.get("http://allevents.in/lahore/")
soup = bs4.BeautifulSoup(res.text)
for link in soup.select('a[property="schema:url"]'):
    print(link.get('href'))

## https://gist.github.com/yanniskatsaros/fb6f163b814b60424d449f6b87fa0545

In [114]:
url = 'https://www.zillow.com/homes/for_sale/78254/0_singlestory/days_sort'
req = requests.get(url)
req.text

'<html><head><meta name="robots" content="noindex, nofollow"/><link href="https://www.zillowstatic.com/vstatic/80d5e73/static/css/z-pages/captcha.css" type="text/css" rel="stylesheet" media="screen"/><script>\n        window._pxAppId = \'PXHYx10rg3\';\n        window._pxJsClientSrc = \'/HYx10rg3/init.js\';\n        window._pxHostUrl = \'/HYx10rg3/xhr\';\n        window._pxFirstPartyEnabled = true;\n        window._pxreCaptchaTheme=\'light\';\n    </script><script type="text/javascript" src="https://captcha.px-cdn.net/PXHYx10rg3/captcha.js?a=c&amp;m=0"></script>\n    <script>\n        function getQueryString(name, url) {\n            if (!url) url = window.location.href;\n            name = name.replace(/[\\[\\]]/g, "\\\\$&");\n            var regex = new RegExp("[?&]" + name + "(=([^&#]*)|&|#|$)"),\n                results = regex.exec(url);\n            if (!results) return null;\n            if (!results[2]) return \'\';\n            return decodeURIComponent(results[2].replace(/\\+/